In [1]:
%matplotlib inline

import urllib.request
import io
import zipfile

from pathlib import Path
import os

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

import igraph as ig

In [2]:
#instance

inst_="karate_club"
G = nx.karate_club_graph()

#inst_="sudoku"
#G = nx.sudoku_graph(5)

#inst_="random_tree"
#G = nx.random_tree(50)

#inst_="random_internet_100"
#G = nx.random_internet_as_graph(100)

# erdos_renyi_graph(n, p, seed=None, directed=False)
#G = nx.erdos_renyi_graph(50,0.1)

# barabasi_albert_graph(n, m, seed=None, initial_graph=None)
#G = nx.barabasi_albert_graph()


#print("Node Degree")
#for v in G:
#    print(f"{v:4} {G.degree(v):6}")
#print(G)

In [3]:
nx.is_connected(G)

True

In [6]:
N = nx.number_of_nodes(G)
M = nx.number_of_edges(G)

dm = np.zeros((N,N), dtype=int)

p = nx.shortest_path(G)

for i in range(0,N):
    for j in range(0,N):
        dm[i][j] = len(p[i][j])-1

#G.edges


In [7]:
# rltForm1
form_="rltForm1"
method_="mip"

N = nx.number_of_nodes(G)
M = nx.number_of_edges(G)

lb = np.zeros((N), dtype=float)
ub = np.zeros((N), dtype=float)
time = np.zeros((N), dtype=float)
gap = np.zeros((N), dtype=float)
nodes = np.zeros((N), dtype=float)
status = np.zeros((N), dtype=float)

RESULT_PATH   = Path('../result/')

for i in G:

    model = gp.Model(f"{method_}_{inst_}_{form_}")

    # configurando parametros
    # model.Params.TimeLimit = 60
    model.Params.MIPGap = 1.e-6
    model.Params.Threads = 1
    # model.Params.Presolve = 0
    # model.Params.Cuts = 0


    if (method_=="mip"):
        x = model.addVars(N, vtype=GRB.BINARY, name="x")
        #x = []
        #for u in range(0, N):
        #   x.append(m.addVar(vtype=GRB.BINARY,name="x_{}".format(u+1)))
    else:
        x = model.addVars(N, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="x")

    #l = list(tuple())
    #for u in range(0, N):
    #    for w in range(u+1, N):
    #        l.append((u,w))

    #y = model.addVars(l, vtype=GRB.BINARY, name='y')

    obj = 0
    for u in range(N):
        obj += 1 * x[u]
        #for w in range(u+1,N):
        #    obj += y[u,w]
         
    model.setObjective(obj, GRB.MAXIMIZE)
     
    # Turn off display and heuristics
    #gp.setParam('OutputFlag', 0)
    #gp.setParam('Heuristics', 0)

    constr0 = 0
    constr0 += 1 * x[i]
    model.addConstr(constr0 == 0)

    for u in range(0,N):
        for w in range(u+1,N):
            if dm[u,w] <= N:
                for s in range(0,N):
                    if (s != u) and (s != w):
                        if (dm[u,s] + dm[s,w] == dm[u,w]):
                            constr1 = x[u] + x[w]
                            model.addConstr(constr1 <= 1 + x[s])

#    for u in range(0,N):
#        for w in range(u+1,N):
#            if (u,w) in G.edges: 
#                constrRlt1 = 1
#                model.addConstr(constrRlt1 <= x[u])

    for e in G.edges:
        (u,w) = e
        model.addConstr(1 <= x[u])

#    for u in range(0,N):
#        for w in range(u+1,N):
#            if (u,w) in G.edges: 
#                constrRlt2 = 1
#                model.addConstr(constrRlt2 <= x[w])

    for e in G.edges:
        (u,w) = e
        model.addConstr(1 <= x[u])

#    for u in range(0,N):
#        for w in range(u+1,N):
#            if (u,w) in G.edges: 
#                constrRlt3 = x[u] + x[w]
#                model.addConstr(constrRlt3 <= 1 + 1)

    for e in G.edges:
        (u,w) = e
        model.addConstr(x[u] + x[w] <= 1 + 1)

    #x[i].Start = 0.0
    #relax = model.relax()
    #relax.optimize()

    model.write(f"{method_}_{inst_}_{form_}.lp")

    model.optimize()

    tmp = 0
    if model.status == GRB.OPTIMAL:
        tmp = 1
    
    #if (method_=="mip"):
    #    lb[i] = N - model.objBound
    #    ub[i] = N - model.objVal
    #    gap[i] = model.MIPGap
    #    time[i] = model.Runtime
    #    nodes[i] = model.NodeCount
    #    status[i] = tmp
    #else:
    #    ub[i] = N - model.objVal
    #    time[i] = model.Runtime
    #    status[i] = tmp

    #if (method_=="mip"):
    #    arquivo = open(os.path.join(RESULT_PATH,f"{method_}_tukey_{inst_}_{form_}.txt"),'a')
    #    tmp = i
    #    arquivo.write(
    #        str(tmp)+';'
    #        +str(round(lb[i],1))+';'
    #        +str(round(ub[i],1))+';'
    #        +str(round(gap[i],2))+';'
    #        +str(round(time[i],2))+';'
    #        +str(round(nodes[i],1))+';'
    #        +str(round(status[i],1))+'\n')
    #    arquivo.close()
    #else:
    #    arquivo = open(os.path.join(RESULT_PATH,f"{method_}_tukey_{inst_}_{form_}.txt"),'a')
    #    tmp = i
    #    arquivo.write(
    #        str(tmp)+';'
    #        +str(round(ub[i],1))+';'
    #        +str(round(time[i],2))+';'
    #        +str(round(status[i],1))+'\n')
    #    arquivo.close()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-01
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 1839 rows, 34 columns and 5125 nonzeros
Model fingerprint: 0x6e0f4295
Variable types: 0 continuous, 34 integer (34 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 78 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 9.5.2 buil

In [ ]:
#t = 0
#for i in ub:
#    print("tukey[%d] =" %t, end=" ")
#    print("%g" %i)
#    t = t+1